# Find Optimal PSF
===========================

- creation 29/06/2016
- author Sylvie Dagoret-Campagne



Find HD158485 with grating

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.modeling import models
from astropy import units as u
from astropy import nddata
from astropy.io import fits

from astropy.table import Table
from astropy.table import Column

import ccdproc
print 'ccdproc version',ccdproc.__version__

from astropy.modeling import models

ccdproc version 1.0.1


In [2]:
import photutils
from astropy.stats import sigma_clipped_stats
from photutils import daofind
from photutils import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

In [3]:
from scipy import stats 
from scipy import ndimage
import os
from datetime import datetime, timedelta

In [4]:
from photutils.background import Background2D

In [5]:
import libMonocamBaseImages           # my tool library written to do that CCD reduction

In [6]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

standard date format for the analysis : 2016-07-06 11:34:53.248142
fits date format for the analysis :  2016-07-06T11:34:53


## Definitions and Constants

In [7]:
object_name='HD158485_grat'

### input files

In [8]:
path='./HD158485_grat'
rootfilename='AssScImHD158485_grat_' 
#NumStart=1
#NumStop=50

#NumStart=51
#NumStop=100

NumStart=1
NumStop=25

### output file (table)

In [9]:
outputtablefile='HD158485_grat_1-25_TablePSF.fits'

### make the filelist

In [10]:
filelist=libMonocamBaseImages.BuildFilelist(path,rootfilename,start=NumStart,stop=NumStop,nbchar=1)

In [11]:
filelist

['./HD158485_grat/AssScImHD158485_grat_1.fits',
 './HD158485_grat/AssScImHD158485_grat_2.fits',
 './HD158485_grat/AssScImHD158485_grat_3.fits',
 './HD158485_grat/AssScImHD158485_grat_4.fits',
 './HD158485_grat/AssScImHD158485_grat_5.fits',
 './HD158485_grat/AssScImHD158485_grat_6.fits',
 './HD158485_grat/AssScImHD158485_grat_7.fits',
 './HD158485_grat/AssScImHD158485_grat_8.fits',
 './HD158485_grat/AssScImHD158485_grat_9.fits',
 './HD158485_grat/AssScImHD158485_grat_10.fits',
 './HD158485_grat/AssScImHD158485_grat_11.fits',
 './HD158485_grat/AssScImHD158485_grat_12.fits',
 './HD158485_grat/AssScImHD158485_grat_13.fits',
 './HD158485_grat/AssScImHD158485_grat_14.fits',
 './HD158485_grat/AssScImHD158485_grat_15.fits',
 './HD158485_grat/AssScImHD158485_grat_16.fits',
 './HD158485_grat/AssScImHD158485_grat_17.fits',
 './HD158485_grat/AssScImHD158485_grat_18.fits',
 './HD158485_grat/AssScImHD158485_grat_19.fits',
 './HD158485_grat/AssScImHD158485_grat_20.fits',
 './HD158485_grat/AssScImHD15

## Read Input files

In [12]:
allchannelallsciimages = []  # list of 16 lists of images series 
exposures_list = []        # sequential list of the exposures of the sky flats 
header_list = []           # list of headers
data_list = []
time_list = []                # date and time
basefile_list = []         # basefilename
dateobs_list = [] 
# get the primary block headers:
for image_file in filelist: 
    print image_file
    hdu_list = fits.open(image_file)
    basefile_list.append(os.path.basename(image_file))
    header=hdu_list[0].header
    exposure=header['EXPOSURE']
    exposures_list.append(exposure)
    dateobs_list.append(header['DATE-OBS'])
    header_list.append(header)
    data=ccdproc.CCDData.read(image_file, hdu=0,unit='adu') 
    data_list.append(data)

./HD158485_grat/AssScImHD158485_grat_1.fits
./HD158485_grat/AssScImHD158485_grat_2.fits
./HD158485_grat/AssScImHD158485_grat_3.fits
./HD158485_grat/AssScImHD158485_grat_4.fits
./HD158485_grat/AssScImHD158485_grat_5.fits
./HD158485_grat/AssScImHD158485_grat_6.fits
./HD158485_grat/AssScImHD158485_grat_7.fits
./HD158485_grat/AssScImHD158485_grat_8.fits
./HD158485_grat/AssScImHD158485_grat_9.fits
./HD158485_grat/AssScImHD158485_grat_10.fits
./HD158485_grat/AssScImHD158485_grat_11.fits
./HD158485_grat/AssScImHD158485_grat_12.fits
./HD158485_grat/AssScImHD158485_grat_13.fits
./HD158485_grat/AssScImHD158485_grat_14.fits
./HD158485_grat/AssScImHD158485_grat_15.fits
./HD158485_grat/AssScImHD158485_grat_16.fits
./HD158485_grat/AssScImHD158485_grat_17.fits
./HD158485_grat/AssScImHD158485_grat_18.fits
./HD158485_grat/AssScImHD158485_grat_19.fits
./HD158485_grat/AssScImHD158485_grat_20.fits
./HD158485_grat/AssScImHD158485_grat_21.fits
./HD158485_grat/AssScImHD158485_grat_22.fits
./HD158485_grat/Ass

In [13]:
#basefile_list

## For control

uncomment for control

In [14]:
index=0

In [15]:
#print exposures_list[index]

In [16]:
#header_list[index]

In [17]:
#plt.imshow(data_list[index])

In [18]:
#bkg= Background2D(data_list[index], (100, 100), filter_size=(3, 3),method='median')

In [19]:
#data_list[index].data-bkg.background

## Background subtraction

In [20]:
correctedimage_list = []

In [21]:
for data in data_list:
    bkg= Background2D(data, (100, 100), filter_size=(3, 3),method='median')
    newimage=data-bkg.background
    correctedimage_list.append(newimage)

## Calculation of PSF

In [22]:
NBIMAGES=len(correctedimage_list)

In [23]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (average, np.sqrt(variance))

In [24]:
image_psf=np.zeros((NBIMAGES,6))

In [25]:
DELTA_NBINSX=10
DELTA_NBINSY=10

In [26]:
# loop on images
index=-1
for image in correctedimage_list:
    index=index+1
    mean, median, std = sigma_clipped_stats(image, sigma=10.0, iters=5) 
    sources = daofind(image - median, fwhm=3.0, threshold=50.*std)
    selected_stars=sources.as_array()
    NBSTARS=selected_stars.shape[0]
    print 'image {} ==> NBSTARS = {}'.format(index,NBSTARS)
    image_psf[index,0]=index
    image_psf[index,1]=NBSTARS
    star_psfx=np.zeros(NBSTARS)
    star_psfy=np.zeros(NBSTARS)
    # loop on stars
    for istar in range(NBSTARS):
        X = int(selected_stars[istar][1])
        Y = int(selected_stars[istar][2])
        prf_image = image[Y-DELTA_NBINSY:Y+DELTA_NBINSY,X-DELTA_NBINSX:X+DELTA_NBINSX]
        oneprfX=prf_image.sum(axis=0)
        oneprfY=prf_image.sum(axis=1)
        if oneprfX.sum() == 0 or oneprfY.sum() == 0:
            star_psfx[istar]=0
            star_psfy[istar]=0
        else:
            posX,sigX=weighted_avg_and_std(np.arange(oneprfX.shape[0]),oneprfX)
            posY,sigY=weighted_avg_and_std(np.arange(oneprfY.shape[0]),oneprfY)
            star_psfx[istar]=sigX
            star_psfy[istar]=sigY
        
    all_sigx=star_psfx[np.logical_not(np.isnan(star_psfx))]
    all_sigy=star_psfy[np.logical_not(np.isnan(star_psfy))]
    all_sigx=all_sigx[all_sigx>2.4]
    all_sigy=all_sigy[all_sigy>2.4]
    print 'average prf(x) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigx),all_sigx.std(),np.median(all_sigx)*0.4*2.36,all_sigx.std()*0.4*2.26)
    print 'average prf(y) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigy),all_sigy.std(),np.median(all_sigy)*0.4*2.36,all_sigy.std()*0.4*2.26)
    image_psf[index,2]=np.median(all_sigx)
    image_psf[index,3]=np.median(all_sigy)
    image_psf[index,4]=all_sigx.std()
    image_psf[index,5]=all_sigy.std()

image 0 ==> NBSTARS = 21
average prf(x) = 4.52 +/- 0.71 pixels ==> psf  4.27 +/- 0.65 arcsec 
average prf(y) = 4.41 +/- 0.47 pixels ==> psf  4.17 +/- 0.43 arcsec 


/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in sqrt


image 1 ==> NBSTARS = 15
average prf(x) = 4.37 +/- 0.72 pixels ==> psf  4.13 +/- 0.65 arcsec 
average prf(y) = 4.56 +/- 0.44 pixels ==> psf  4.30 +/- 0.40 arcsec 
image 2 ==> NBSTARS = 12
average prf(x) = 4.64 +/- 0.37 pixels ==> psf  4.38 +/- 0.33 arcsec 
average prf(y) = 4.15 +/- 0.46 pixels ==> psf  3.92 +/- 0.42 arcsec 
image 3 ==> NBSTARS = 23
average prf(x) = 4.63 +/- 0.43 pixels ==> psf  4.37 +/- 0.39 arcsec 
average prf(y) = 4.32 +/- 0.42 pixels ==> psf  4.08 +/- 0.38 arcsec 
image 4 ==> NBSTARS = 19
average prf(x) = 4.82 +/- 0.62 pixels ==> psf  4.55 +/- 0.56 arcsec 
average prf(y) = 4.22 +/- 0.49 pixels ==> psf  3.98 +/- 0.45 arcsec 
image 5 ==> NBSTARS = 18
average prf(x) = 4.54 +/- 0.34 pixels ==> psf  4.28 +/- 0.31 arcsec 
average prf(y) = 4.17 +/- 0.35 pixels ==> psf  3.93 +/- 0.32 arcsec 
image 6 ==> NBSTARS = 14
average prf(x) = 4.51 +/- 0.42 pixels ==> psf  4.26 +/- 0.38 arcsec 
average prf(y) = 4.50 +/- 0.37 pixels ==> psf  4.25 +/- 0.34 arcsec 
image 7 ==> NBSTARS = 

## Create astropy table

In [27]:
t=Table(rows=image_psf,names=('num','nbstars','prfx','pfry','sig_prfx','sig_prfy'),dtype=('i4','i4','f8','f8','f8','f8'))

In [28]:
t

num,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,int32,float64,float64,float64,float64
0,21,4.51917437069,4.41227611476,0.714686734982,0.470638109358
1,15,4.37330406145,4.55735594953,0.724104853084,0.439038971578
2,12,4.63608968996,4.14849632127,0.368357486235,0.460349718574
3,23,4.63124096986,4.31863744567,0.427319048562,0.422820799627
4,19,4.82428803566,4.2211565421,0.618518011995,0.492391423441
5,18,4.53751815256,4.16726183081,0.341235964668,0.349962604607
6,14,4.51257238741,4.4991794035,0.421523140932,0.37097916161
7,11,4.38819420274,4.442448533,0.308672418031,0.262399443441
8,25,4.68053308095,4.44502209162,0.379893380493,0.444990478455


In [29]:
expo = Column(exposures_list, name='exposure')
file = Column(basefile_list, name='file')
time = Column(dateobs_list,name='time')

In [30]:
t.add_column(expo, index=1)
t.add_column(time, index=1)
t.add_column(file, index=1)

In [31]:
t

num,file,time,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,str28,str23,float64,int32,float64,float64,float64,float64
0,AssScImHD158485_grat_1.fits,2016-05-10T08:16:06.228,1.0,21,4.51917437069,4.41227611476,0.714686734982,0.470638109358
1,AssScImHD158485_grat_2.fits,2016-05-10T08:17:42.514,1.5,15,4.37330406145,4.55735594953,0.724104853084,0.439038971578
2,AssScImHD158485_grat_3.fits,2016-05-10T08:20:57.045,1.5,12,4.63608968996,4.14849632127,0.368357486235,0.460349718574
3,AssScImHD158485_grat_4.fits,2016-05-10T08:21:03.531,1.5,23,4.63124096986,4.31863744567,0.427319048562,0.422820799627
4,AssScImHD158485_grat_5.fits,2016-05-10T08:21:09.092,1.5,19,4.82428803566,4.2211565421,0.618518011995,0.492391423441
5,AssScImHD158485_grat_6.fits,2016-05-10T10:37:40.318,1.5,18,4.53751815256,4.16726183081,0.341235964668,0.349962604607
6,AssScImHD158485_grat_7.fits,2016-05-10T10:38:22.323,1.5,14,4.51257238741,4.4991794035,0.421523140932,0.37097916161
7,AssScImHD158485_grat_8.fits,2016-05-10T10:38:57.336,1.5,11,4.38819420274,4.442448533,0.308672418031,0.262399443441
8,AssScImHD158485_grat_9.fits,2016-05-10T10:39:04.394,1.5,25,4.68053308095,4.44502209162,0.379893380493,0.444990478455


## Write output file

In [32]:
t.write(outputtablefile,format='fits')

## Pandas

In [33]:
df=t.to_pandas()

In [34]:
df.describe()

,num,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
count,25.000000,25.00,25.000000,25.000000,25.000000,25.000000,25.000000
mean,12.000000,1.48,20.120000,4.501416,4.345221,0.498199,0.412205
std,7.359801,0.10,4.719816,0.173333,0.179147,0.132740,0.104753
min,0.000000,1.00,11.000000,4.148797,3.663572,0.308672,0.210265
25%,6.000000,1.50,18.000000,4.373304,4.304302,0.421523,0.367556
50%,12.000000,1.50,21.000000,4.509932,4.409222,0.470657,0.417392
75%,18.000000,1.50,24.000000,4.631241,4.456345,0.561964,0.460350
max,24.000000,1.50,30.000000,4.824288,4.557356,0.873762,0.781951
